In [1]:
# Load libraries
import os
import re
import fileinput
import sys
import json

# Create path to content
path = 'content/'

# Find all jupyter notebooks in all content folders
all_ipynb_files = [os.path.join(root, name)
                   for root, dirs, files in os.walk(path)
                       for name in files
                           if name.endswith((".ipynb"))]

# Remove all notebooks from checkpoint folders
ipynb_files = [ x for x in all_ipynb_files if ".ipynb_checkpoints" not in x ]

In [2]:
for file in ipynb_files:
    with open(file) as data_file:    
        data = json.load(data_file)
        
        if data['cells'][0]['cell_type'] == 'markdown':
            data['cells'][0]['cell_type'] = 'raw'

            old_frontmatter = data['cells'][0]['source']

            for row in old_frontmatter:
                if "Title:" in row:
                    title = row.split(":")[1].split("\n")[0].strip()
                if "Summary:" in row:
                    description = row.split(":")[1].split("\n")[0].strip()

            title_string = f"title: \"{title}\"\n".replace('\'', '')
            description_string = f"description: \"{description}\"\n".replace('', '')

            new_frontmatter = ["---\n",
                                title_string,
                                "author: \"Chris Albon\"\n",
                                "date: 2017-12-20T11:53:49-07:00\n",
                                 description_string,
                                "type: technical_note\n",
                                "draft: false\n",
                                "---"]

            data['cells'][0]['source'] = new_frontmatter
            
    with open(file, 'w') as outfile:
        json.dump(data, outfile)